In [ ]:
import spotipy
import time
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import sys
import requests
#mport requests_oauthlib
import plotly.express as px

In [ ]:
client_id = "5df1ce77a68942fe8c89e8da7cf4e177"
client_secret = "7e22edf656734c9ab13c13859493a70c"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [ ]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('Sebastian Kachniarz','23RSJQkb2GqNeCSi0fx7gX')

In [ ]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    
#features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    
    track = [name, album, artist, release_date, length, popularity, acousticness,  danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

In [ ]:
tracks = []
for i in range(len(ids)):
    track = getTrackFeatures(ids[i])
    tracks.append(track)

df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("Spotify_API.csv", sep=',')
df

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import countDistinct

spark = SparkSession.builder.appName("app").getOrCreate()
df_spark = spark.read.option("header", "True").csv('Spotify_API.csv', sep=",")
df_spark.createOrReplaceTempView("spotify_data")

In [ ]:
df_spark

In [ ]:
#Few rank tables using sql sentences...

spark.sql('''
select first(artist) artist, name, round(avg(length)/60000,2) length
from spotify_data
group by name
order by length desc
limit 20
''').show()

In [ ]:
spark.sql('''
select first(artist) artist, name, popularity
from spotify_data
order by popularity desc
limit 20
''').show()

In [ ]:
spark.sql('''
select first(artist) artist, name, release_date
from spotify_data
order by release_date asc
limit 20
''').show()

In [ ]:
spark.sql('''
select count(name)
from spotify_data
''').show()

In [ ]:
fig = px.pie(df, values='length', names='artist')
fig.show()

In [ ]:
df[df['artist']=='[]']
data_artist = df[df['artist']!='[]']
data_artist.head(10)

In [ ]:
df_sort_artist = pd.DataFrame(data_artist.assign(artist=data_artist.artist.str.split(",")).explode('artist'))
df_sort_artist.tail(10)

In [ ]:
df_sort_artist['artist'] = df_sort_artist.artist.str.replace('[',' ')
df_sort_artist['artist'] = df_sort_artist.artist.str.replace(']',' ')

In [ ]:
n = 30
top_n = pd.DataFrame(df_sort_artist['artist'].value_counts()[:n]).reset_index()
top_n.rename(columns = {'index':'Artist','artist':'Total_Count'}, inplace = True)
top_n

In [ ]:
fig = px.pie(top_n, values='Total_Count', names='Artist')
fig.show()

In [ ]:
fig2 = px.scatter(df, x='popularity', y='danceability')
fig2.show()